In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## class definitions

In [2]:
class RansacPotDetector:
    def __init__(self):
        pass

    def _preprocess(self, pcd : o3d.geometry.PointCloud, voxel_size: float):
        pcd_down_sampled = pcd.voxel_down_sample(voxel_size)
        pcd_down_sampled.normal_estimate()
        
        return pcd_down_sampled

    def _extract_fpfh_features(self, pcd : o3d.geometry.PointCloud, voxel_size: float):
        fpfh_search_radius = 5 * voxel_size # default is 5
        fpfh_max_nearest_neighbor = 30 # 100 is good for paprika base
        
        fpfh_features =  o3d.pipelines.registration.compute_fpfh_feature(
            pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=fpfh_search_radius, max_nn=fpfh_max_nearest_neighbor)
        )

        return fpfh_features

    def _ransac_registration(self, pcd_source: o3d.geometry.PointCloud, pcd_target: o3d.geometry.PointCloud, 
                             fpfh_source, fpfh_target, voxel_size: float):
        distance_threshold = 1.5 * voxel_size # default is 1.5
        estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPoint(False) # disable scalling
        use_ratio_test = True
        enable_checkers = [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.8),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold),
        ]
        criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)

        ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            pcd_source, pcd_target, fpfh_source, fpfh_target, use_ratio_test, distance_threshold,
            estimation_method, ransac_n=3, checkers=enable_checkers
        )
        
        return ransac_result
        
    def detect(self, pcd_source, pcd_target, voxel_size=0.005, visualize=False):
        # down sampling
        pcd_source_down_sampled = pcd_source.voxel_down_sample(voxel_size)
        pcd_target_down_sampled = pcd_target.voxel_down_sample(voxel_size)
        
        ## normals estimation
        # this makes normals look inside (TODO : make nomals look outside)
        pcd_source_down_sampled.estimate_normals()
        pcd_source_down_sampled.orient_normals_consistent_tangent_plane(k=15)
        pcd_target_down_sampled.estimate_normals()
        pcd_target_down_sampled.orient_normals_consistent_tangent_plane(k=15)

        # fpfh feature extraction
        fpfh_source = self._extract_fpfh_features(pcd_source_down_sampled, voxel_size)
        fpfh_target = self._extract_fpfh_features(pcd_target_down_sampled, voxel_size)

        # ransac
        result = self._ransac_registration(pcd_source_down_sampled, pcd_target_down_sampled,
                                         fpfh_source, fpfh_target, voxel_size)

        # display the ransac result
        if visualize == True:
            pcd_source_down_sampled.paint_uniform_color([1, 0, 0])
            pcd_target_down_sampled.paint_uniform_color([0, 1, 0])
            self.draw_registration_result([pcd_source_down_sampled], [pcd_target_down_sampled], result.transformation, window_name='ransac result')
        
        return result

    def draw_registration_result(self, source, target, transformation, window_name='open3d'):
        pcds = list()
        for s in source:
            temp = copy.deepcopy(s)
            pcds.append(temp.transform(transformation))
        pcds += target
        o3d.visualization.draw_geometries(pcds, window_name)

In [3]:
class ICPPotDetector:
    def __init__(self):
        pass

    def detect(self, pcd_source, pcd_target, initial_transform, voxel_size:float=0.005, distance_threshold:float=0.2, visualize:bool=False):
        # down sampling
        pcd_source_down_sampled = pcd_source.voxel_down_sample(voxel_size)
        pcd_target_down_sampled = pcd_target.voxel_down_sample(voxel_size)

        # icp
        icp_criteria = o3d.pipelines.registration.ICPConvergenceCriteria(
            max_iteration=100000,
            relative_fitness=1e-6,
            relative_rmse=1e-6
        )

        result = o3d.pipelines.registration.registration_icp(
            pcd_source_down_sampled, pcd_target_down_sampled, distance_threshold, initial_transform,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
            #o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            criteria=icp_criteria
        )

        # display result
        print(f'||| visualize : {visualize, distance_threshold, voxel_size}')
        if visualize == True:
            pcd_source_down_sampled.paint_uniform_color([1, 0, 0])
            pcd_target_down_sampled.paint_uniform_color([0, 1, 0])
            self.draw_registration_result([pcd_source_down_sampled], [pcd_target_down_sampled], result.transformation, window_name='icp result')

        return result

    def draw_registration_result(self, source, target, transformation, window_name='open3d'):
        pcds = list()
        for s in source:
            temp = copy.deepcopy(s)
            pcds.append(temp.transform(transformation))
        pcds += target
        o3d.visualization.draw_geometries(pcds, window_name)

In [4]:
class HelperCollection:
    def __init__(self):
        import copy

    def load_realsense_pcd(file_path='../dataset/d405/d405-1.ply', visualize = False):
        print(f'||| loading {file_path} ...')
        pcd_source = o3d.io.read_point_cloud(file_path)
        print(f'||| loading complete')
        
        if visualize == True:
            mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0.0,0.0,0.0])
            o3d.visualization.draw_geometries([pcd_source, mesh_frame])

        return pcd_source

    def load_orbbec_pcd(file_path, visualize=False):
        print(f'||| loading {file_path}')
        pcd_source = o3d.io.read_point_cloud(file_path)
        print(f'||| loading complete')

        # scale mm to m (1/1000)
        pcd_source.scale(0.001, np.array([0,0,0]))

        # normal estimation
        pcd_source.estimate_normals()
        pcd_source.orient_normals_towards_camera_location()

        if visualize == True:
            frame_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0,0,0])
            o3d.visualization.draw_geometries([pcd_source, frame_mesh])

        return pcd_source
    
    def load_cad_model_pcd(file_path='../dataset/model/art_melon_without_handle.STL', visualize = False):          
        print(f'||| loading {file_path}')
        mesh_target = o3d.io.read_triangle_mesh(file_path)
        print(f'||| loading complete')
        mesh_target.compute_triangle_normals()
        pcd_target = mesh_target.sample_points_poisson_disk(number_of_points=50000)
        
        if visualize == True:
            mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
            o3d.visualization.draw_geometries([mesh_target, mesh_frame])

        return pcd_target

    def crop_arround_pot(pcd_source):
        tmp_pcd_source = copy.deepcopy(pcd_source)
        
        # crop only bottle
        min_bound = np.asarray([-0.5, -1.0, -1.0]) # [x_min, y_min, z_min]
        max_bound = np.asarray([0.25, 1.0, 1.0]) # [x_max, y_max, z_max]
        aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
        pcd_source_cropped = tmp_pcd_source.crop(aabb)
        print(f'||| croping arround pot by aabb')
        mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0,0,0])
        o3d.visualization.draw_geometries([pcd_source_cropped, mesh_frame])

        # remove table
        plane_model, plane_inlier_indeces = pcd_source_cropped.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=500)
        pcd_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces)
        pcd_plane.paint_uniform_color([1, 0, 0])
        o3d.visualization.draw_geometries([pcd_source_cropped, pcd_plane])
        pcd_source_removed_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces, invert=True)
        print('||| estimate plane and remove it')
        o3d.visualization.draw_geometries([pcd_source_removed_plane, mesh_frame])

        # remove outlier
        cl, inlier_indeces = pcd_source_removed_plane.remove_statistical_outlier(nb_neighbors=60, std_ratio=2.5)
        pcd_source_removed_outlier = pcd_source_removed_plane.select_by_index(inlier_indeces)
        print('||| remove outlier')
        o3d.visualization.draw_geometries([pcd_source_removed_outlier, mesh_frame])

        return pcd_source_removed_outlier

    def crop_arround_dura_bright(pcd_source, visualize=False):
        tmp_pcd_source = copy.deepcopy(pcd_source)

        # crop only dura bright
        min_bound = np.asarray([-1, -1, -1])
        max_bound = np.asarray([1, 1, 1])
        aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
        pcd_source_cropped = tmp_pcd_source.crop(aabb)
        print(f'||| cropping arround dura bright by aabb')

        if visualize == True:
            mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0,0,0])
            o3d.visualization.draw_geometries([pcd_source_cropped, mesh_frame])

        # remove table
        plane_model, plane_inlier_indeces = pcd_source_cropped.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=500)
        pcd_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces)
        pcd_plane.paint_uniform_color([1, 0, 0])
        o3d.visualization.draw_geometries([pcd_source_cropped, pcd_plane])
        pcd_source_removed_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces, invert=True)
        print('||| estimate plane and remove it')
        o3d.visualization.draw_geometries([pcd_source_removed_plane, mesh_frame])
        #pcd_source_removed_plane = pcd_source_cropped

        # remove outlier
        cl, inlier_indeces = pcd_source_removed_plane.remove_statistical_outlier(nb_neighbors=60, std_ratio=2.5)
        pcd_source_removed_outlier = pcd_source_removed_plane.select_by_index(inlier_indeces)
        print('||| remove outlier')
        o3d.visualization.draw_geometries([pcd_source_removed_outlier, mesh_frame])

        return pcd_source_removed_outlier

    def rotate_y_axis_to_upper(pcd_cad_model: o3d.geometry.PointCloud):
        tmp_pcd_cad_model = copy.deepcopy(pcd_cad_model)
        
        rad = np.radians(-45) # 90deg for art melon
        R = o3d.geometry.get_rotation_matrix_from_axis_angle(np.array([rad, 0, 0]))
        tmp_pcd_cad_model.rotate(R)
        
        return tmp_pcd_cad_model

    def move_centroid_to_origin(pcd):
        tmp_pcd = copy.deepcopy(pcd) 
        tmp_pcd.translate(-tmp_pcd.get_center())
        return tmp_pcd

## test ransac class

#### 1. load data

In [5]:
# load cad model as pcd
#file_path = '../dataset/model/art_melon_without_handle.STL'
#file_path = '../dataset/model/dura_bright.STL'
file_path = '../dataset/model/paprika_base.STL'
pcd_target = HelperCollection.load_cad_model_pcd(file_path, visualize=True)

||| loading ../dataset/model/paprika_base.STL
||| loading complete


In [6]:
# load realsense ovservation as pcd
#file_path = '../dataset/d405/art_melon/d405-1.ply'
#file_path = '../dataset/d405/dura_bright/1.ply'
#file_path = '../dataset/d405/paprika_base/1.ply'
#pcd_source = HelperCollection.load_realsense_pcd(file_path, visualize=True)

In [7]:
# load orbbec ovservation as pcd
file_path = '../dataset/orbbec/pointcloud/1769153567114890/RGBDPoints_1769153567114890.ply'
pcd_source = HelperCollection.load_orbbec_pcd(file_path, visualize=True)

||| loading ../dataset/orbbec/pointcloud/1769153567114890/RGBDPoints_1769153567114890.ply
||| loading complete


#### 2. preprocess (rotate and so on)

In [8]:
# crop arround pot
pcd_source_cropped = HelperCollection.crop_arround_dura_bright(pcd_source, visualize=True)

||| cropping arround dura bright by aabb
||| estimate plane and remove it
||| remove outlier


In [9]:
# rotate the cad model -90deg
#pcd_target_rotated = HelperCollection.rotate_y_axis_to_upper(pcd_target)
#pcd_target_aligned = HelperCollection.move_centroid_to_origin(pcd_target_rotated)
#mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0,0,0])
#o3d.visualization.draw_geometries([pcd_target_aligned, mesh_frame])

In [10]:
# move the cad model centroid to observation centroid
#pcd_target_aligned.translate(pcd_source_cropped.get_center())
#o3d.visualization.draw_geometries([pcd_target_aligned, pcd_source_cropped, mesh_frame])

#### 3. ransac

In [11]:
ransac_detector = RansacPotDetector()

In [12]:
# 0.01 for art-melon pot
voxel_size = 0.005
ransac_result = ransac_detector.detect(pcd_source_cropped, pcd_target, voxel_size, visualize=True)

[Open3D WARNING] Too few correspondences (184) after mutual filter, fall back to original correspondences.


In [13]:
ransac_result.transformation

array([[ 0.44074376, -0.73810204,  0.51083296, -0.30267175],
       [-0.89579404, -0.3252619 ,  0.30291538, -0.18369531],
       [-0.05742797, -0.59110918, -0.80454457,  0.68880877],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

#### 4. icp (by ransac initial transform guess)

In [14]:
icp_detector = ICPPotDetector()

In [15]:
distance_threshold = 0.2 # 0.2m for art melon
voxel_size = 0.005 # 1cm
icp_result1 = icp_detector.detect(pcd_source_cropped, pcd_target, ransac_result.transformation, voxel_size, distance_threshold, True)

||| visualize : (True, 0.2, 0.005)


In [16]:
pcd_source1 = pcd_source_cropped
pcd_target1 = pcd_target

#### 5. icp (by aligned centroid)

In [ ]:
o3d.visualization.draw_geometries([pcd_source_cropped, pcd_target], window_name='initial_state_of_icp2')

In [ ]:
distance_threshold = 0.4 # 0.2m for art melon
voxel_size = 0.01 # 1cm
initial_transform = np.eye(4)

In [ ]:
print(pcd_source_cropped.get_center())
print(pcd_target.get_center())

In [ ]:
pcd_target.translate(-pcd_target.get_center())
pcd_source_cropped.translate(-pcd_source_cropped.get_center())
o3d.visualization.draw_geometries([pcd_source_cropped, pcd_target])

icp_result2 = icp_detector.detect(pcd_source_cropped, pcd_target, initial_transform, voxel_size, distance_threshold, True)

In [ ]:
icp_result2.transformation

## visualize

#### visualize icp with ransac

In [20]:
pcd_target.transform(icp_result1.transformation)

PointCloud with 50000 points.

In [21]:
o3d.visualization.draw_geometries([pcd_source1, pcd_target1, pcd_source])

In [ ]:
def draw_registration_result(source, target, transformation, window_name='open3d'):
    pcds = list()
    for s in source:
        temp = copy.deepcopy(s)
        pcds.append(temp.transform(transformation))
    pcds += target
    o3d.visualization.draw_geometries(pcds, window_name)

#### visualize icp with controid align

In [ ]:
transform = icp_result2.transformation

In [ ]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0,0,0])
o3d.visualization.draw_geometries([pcd_target, mesh_frame])

In [ ]:
def draw_registration_result(source, target, transformation, window_name='open3d'):
    pcds = list()
    for s in source:
        temp = copy.deepcopy(s)
        pcds.append(temp.transform(transformation))
    pcds += target
    o3d.visualization.draw_geometries(pcds, window_name)

In [ ]:
target_mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=transform[:3, 3]+pcd_source_cropped.get_center())
draw_registration_result([pcd_source], [pcd_target, mesh_frame, target_mesh_frame], transform)

In [ ]:
draw_registration_result([pcd_source], [pcd_target, mesh_frame], transform)